In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import  VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import  Dense,Flatten,GlobalAveragePooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


# load data
train_dir = '/content/drive/MyDrive/shoe/train'
test_dir = '/content/drive/MyDrive/shoe/test'
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base layers (so pretrained features stay fixed):
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(4096, activation='relu')(x)
x= Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(1, activation='sigmoid')(x)





2026-02-23 15:28:47.212148: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-23 15:28:47.214290: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-23 15:28:47.217214: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-23 15:28:47.225981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771838927.240786  230164 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771838927.24

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/shoe/train'

In [ ]:
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, epochs=20, validation_data=test_data,verbose=1)

loss,acc=model.evaluate(test_data)
print(f'loss:{loss} acc:{acc}')


In [ ]:
plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tensorflow.keras.preprocessing import image
class_names = list(test_data.class_indices.keys())
print("Classes:", class_names)

# Get all image file paths
all_images = []
for cls in class_names:
    folder = os.path.join(test_dir, cls)
    for img_name in os.listdir(folder):
        all_images.append((os.path.join(folder, img_name), cls))

# Pick 5 random images
samples = random.sample(all_images, 15)

# Plot and predict
plt.figure(figsize=(15, 8))

for i, (img_path, true_label) in enumerate(samples):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_preprocessed = tf.keras.applications.vgg16.preprocess_input(img_array)
    img_expanded = np.expand_dims(img_preprocessed, axis=0)

    # Predict
    pred = model.predict(img_expanded)

    pred_label = class_names[int(pred > 0.5)]
    #
    # Show
    plt.subplot(3, 5, i+1)
    plt.imshow(np.uint8(img))
    plt.axis('off')
    plt.title(f"Pred: {pred_label}\nTrue: {true_label}")

plt.tight_layout()
plt.show()